# Theano Backend Derivative Calculation Dev Portal 

In [8]:
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda0,floatX=float64,dnn.enabled=True"

import keras.backend as K


In [29]:
import theano

## 0. Test Data Loading
This section loads the test symmetry function data set, and load the neural network just to test whether the IO part is working propertly. 

In [9]:
import ModelIO

In [10]:
import Utils.DirNav 

In [11]:
project_dir = Utils.DirNav.get_project_dir()
glycine_dir = os.path.join(project_dir, 'model', 'glycine_opt')
print(glycine_dir)

/home/junmian/Applications/YeltsinNet/model/glycine_opt


In [13]:
h_subnet = ModelIO.read_keras_subnet_ind(glycine_dir, 'H-subnet')

In [15]:
# Import Necessary Test Data
Gfunc_data = ModelIO.read_data_from_pickle(glycine_dir, 'Gfunc_data_test.pkl')

In [38]:
energy_arr = h_subnet.predict(Gfunc_data['H'][3])

In [39]:
energy_arr.shape

(1001, 1)

#### Result: 
Thus, we have proven that Theano backend is capable of loading the weight from tensorflow training set. 

## 1. Copy and Paste Derivative Functions

In [24]:
def get_subnet_deriv(subnet, nb_symm):
    """Generate a dE/dG function for a given elemental sub-neural network.

        Args:
            subnet: keras.model object for the sub-neural network.
            nb_symm: int, number of symmetry functions for the vector.

        Outputs:
            subnet_deriv([G_ind]), a function that takes the symmetry vector of shape (1, nb_symm) of the given atom
            and return the dE/dG matrix for the atom.

    """
    G_tensor_ind = K.placeholder(shape = (1, nb_symm))
    E_tensor_ind = subnet(G_tensor_ind)
    import pdb; pdb.set_trace()
    subnet_deriv = K.function([G_tensor_ind], K.gradients(E_tensor_ind, [G_tensor_ind]))

    return subnet_deriv




In [18]:
Gfunc_data['H'][3].shape

(1001, 276)

In [21]:
G_tensor_ind = K.placeholder(shape = (1, n_symm_func))
E_tensor_ind = h_subnet(G_tensor_ind)
#subnet_deriv = K.function([G_tensor_ind], K.gradients(E_tensor_ind, [G_tensor_ind]))


In [26]:
E_tensor_ind

Elemwise{add,no_inplace}.0

In [27]:
G_tensor_ind

/placeholder

In [33]:
E_tensor_ind[0]

Subtensor{int64}.0

In [35]:
E_tensor_ind.ndim

2

In [40]:
E_tensor_ind[0][0].ndim

0

### Problem: 
The Neural Network is designed to train an ensemble of the same molecule, rather than for individual molecules. Therefore, Theano indicates that we cannot calculate the derivative for many molecules at once. 
<br/>

### Trick:
The only place that we are going to use derivative is (for molecular dynamics simulation), for which we are only going to use derivatives for individual molecules. (Maybe it is also required for the calculations of derivative for the ensemble, which may need another equation, but not for now) The temporary solution is to directly assume that structure of the derivative, and then calculate it. 
<br/>

### Assumption: 
Gfunc_data_ind = (1, 1, n_symm_func) <br/>
energy_predict = (1, 1)   <br/>

_Therefore_ <br/>
energy = energy_predict[0][0]


In [41]:
G_tensor_ind = K.placeholder(shape = (1, n_symm_func))
E_tensor_ind = h_subnet(G_tensor_ind)
E_scalar     = E_tensor_ind[0][0]
h_subnet_deriv = K.function([G_tensor_ind], K.gradients(E_scalar, [G_tensor_ind]))

In [51]:
Gfunc_data['H'][3][0:1,:].shape

(1, 276)

In [52]:
output = h_subnet_deriv([Gfunc_data['H'][3][0:1,:]])

In [53]:
import numpy as np
output = np.array(output)

In [54]:
output.shape

(1, 1, 276)

In [55]:
def get_subnet_deriv_theano(subnet, n_symm_func):
    """Generate a dE/dG function for a given elemental sub-neural network.
    (Specifically For Theano Backend) Only for one specific element, with 
    1 sapmle. 
        

        Args:
            subnet: keras.model object for the sub-neural network.
            nb_symm: int, number of symmetry functions for the vector.

        Outputs:
            subnet_deriv([Gfunc_ind]), a function that takes the symmetry vector of shape (1, nb_symm) of the given atom
            and return the dE/dG matrix for the atom.
            For Gfunc_ind = (1, n_symm_func)
            
        Assuming: 
            G_func_ind has only 1 sample instead of an ensemble of samples. 
            
            
            

    """
    G_tensor_ind = K.placeholder(shape = (1, n_symm_func))
    E_tensor_ind = subnet(G_tensor_ind)
    E_scalar     = E_tensor_ind[0][0]
    subnet_deriv = K.function([G_tensor_ind], K.gradients(E_scalar, [G_tensor_ind]))

    return subnet_deriv



In [56]:
n_symm_func = 276

h_subnet_deriv = get_subnet_deriv_theano(h_subnet, n_symm_func)

In [4]:

def get_subnet_deriv_arr_theano(subnet_list, nb_symm):
    """Generated the array for the dE/dG function.
    !! For Elemental Neural Network only

        Args:
            subnet_list: a list that includes all the sub-neural network with its element
            nb_symm: number of symmetry functions for the vector.

        Outputs:
            a pandas.Serires() array of subnet_deriv([G_ind]), a function that takes the symmetry
            vector of shape (1, nb_symm) of the given atom
            and return the dE/dG matrix for the atom.
    """
    subnet_deriv_arr = {}
    for element, subnet in subnet_list.items():
        subnet_deriv_arr[element] = get_subnet_deriv_theano(subnet, nb_symm)

    return subnet_deriv_arr





In [ ]:
def calculate_subnet_deriv_value(subnet_deriv_arr, element, Gfunc_ind):
    """

            Args:
                Gfunc_ind: Gfunc for individual atoms in the shape
                            (1, symm_func)

            Outputs:
                output: dE(subnet)/dG for the given atom (determined by Gfunc_ind)
                (1,symm_func)


    Assumes that Gfunc_ind has the same No. Symmetry function as the function for dE_dG_arr
    """
    subnet_deriv = subnet_deriv_arr[element]
    output =  np.array(subnet_deriv([Gfunc_ind])).reshape(Gfunc_ind.shape).astype(np.float64)
    return output


## 1. Prototype Theano Functions 